### Libraries import and configurations setup

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import itertools
import numpy as np
import math
import pickle
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

Using CPU


### Class definitions

In [3]:
class MatrixDataset(Dataset):
    def __init__(self, matrices, labels):
        self.matrices = matrices
        self.labels = labels

    def __len__(self):
        return len(self.matrices)

    def __getitem__(self, idx):
        matrix = self.matrices[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return matrix, label
    
class Prey_Net(nn.Module):
    def __init__(self):
        super(Prey_Net, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(9, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 5)

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
class Predator_Net(nn.Module):
    def __init__(self):
        super(Predator_Net, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(9, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 5)

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def find_closest_cell(t, point, n):
    """
    t: torch tensor filled with 1s and 2s
    point: tuple (x,y) of the currently observed point
    n: number to search for (1 or 2)
    """
    closest_dist = math.inf
    closest_cell = None
    
    cells = [(i, j) for i in range(t.size()[0]) for j in range(t.size()[1]) if t[i][j] == n]
    
    for cell_point in cells:
        dist = abs(point[0] - cell_point[0]) + abs(point[1] - cell_point[1]) 
        if dist < closest_dist:
            closest_dist = dist
            closest_cell = cell_point
                    
    if closest_cell is None:
        return math.inf
    else:
        return closest_dist


def find_best_move(t, agent):
# Moves are enumerated as # Stand, Top, Right, Bottom, Left
    n = 1 if agent == 2 else 2
    center = (1,1)

    if agent == 1:
        best_distance = -math.inf
    else:
        best_distance = math.inf

    best_moves = []
    for move in [ (center[0]-1, center[1], 1),  (center[0], center[1]+1, 2), (center[0]+1, center[1], 3), (center[0], center[1]-1, 4)]:
        if (agent == 1 and t[move[0]][move[1]] == 0) or (agent == 2 and t[move[0]][move[1]] != 2):
            new_distance = find_closest_cell(t, move[:2], n)
            
            if (agent == 1 and new_distance > best_distance) or (agent == 2 and new_distance < best_distance):
                best_distance = new_distance
                best_moves = [move[2]]
            elif new_distance == best_distance:
                best_moves.append(move[2])
      

    if len(best_moves) == 0:
        return [1, 0, 0, 0, 0]
      
    prob = 1/len(best_moves)
    prob_vec = [0]*5

    for i in best_moves:
        prob_vec[i] = prob

    return prob_vec


def infer(net, t):
    net.eval()

    with torch.no_grad():  # Disable gradient calculation to save memory and computation
        logits = net(t.unsqueeze(0).to(device))
        probabilities = torch.softmax(logits, dim=1)

        return [el.item() for el in probabilities[0]] 

### State and label generation

In [4]:
# Create a list of all possible combinations of 0, 1, and 2
vals = [0, 1, 2]
combinations = list(itertools.product(vals, repeat=9))

# Filter out the combinations where the middle element is not 0
filtered_combinations = list(filter(lambda x: x[4] == 1, combinations))
prey_tensors = list(set([torch.tensor(combination, dtype=torch.float32).reshape(3, 3) for combination in filtered_combinations]))
prey_labels = [find_best_move(t, 1) for t in prey_tensors]  

filtered_combinations = list(filter(lambda x: x[4] == 2, combinations))
predator_tensors = list(set([torch.tensor(combination, dtype=torch.float32).reshape(3, 3) for combination in filtered_combinations]))
predator_labels = [find_best_move(t, 2) for t in predator_tensors]  

### Prey training

In [5]:
batch_size = 32
dataset = MatrixDataset(prey_tensors, prey_labels)
trainloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

prey_net = Prey_Net().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(prey_net.parameters(), lr=0.001)

num_epochs = 500

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = prey_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch: {epoch + 1}, Loss: {running_loss / len(trainloader)}")

print("Finished training")

Epoch: 1, Loss: 0.8090926532269663
Epoch: 2, Loss: 0.31434288045879705
Epoch: 3, Loss: 0.28651299674823444
Epoch: 4, Loss: 0.274808619713899
Epoch: 5, Loss: 0.26637239349119873
Epoch: 6, Loss: 0.26235832226941885
Epoch: 7, Loss: 0.25455049888312237
Epoch: 8, Loss: 0.2408222678406797
Epoch: 9, Loss: 0.23771723249285184
Epoch: 10, Loss: 0.2355087920639085
Epoch: 11, Loss: 0.23326256528645933
Epoch: 12, Loss: 0.23441157088551706
Epoch: 13, Loss: 0.23279969140212503
Epoch: 14, Loss: 0.22749612591842466
Epoch: 15, Loss: 0.22597890930816542
Epoch: 16, Loss: 0.22424876498729804
Epoch: 17, Loss: 0.2234749282773548
Epoch: 18, Loss: 0.22281770056223235
Epoch: 19, Loss: 0.22003575003235737
Epoch: 20, Loss: 0.2200143055375626
Epoch: 21, Loss: 0.22218192535117992
Epoch: 22, Loss: 0.22247553732499337
Epoch: 23, Loss: 0.21715746360188984
Epoch: 24, Loss: 0.21638571240744797
Epoch: 25, Loss: 0.21712999921591583
Epoch: 26, Loss: 0.21550850100016292
Epoch: 27, Loss: 0.21433850406732374
Epoch: 28, Loss: 

### Predator training

In [6]:
batch_size = 32
dataset = MatrixDataset(predator_tensors, predator_labels)
trainloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

predator_net = Predator_Net().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(predator_net.parameters(), lr=0.001)

num_epochs = 500

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = predator_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch: {epoch + 1}, Loss: {running_loss / len(trainloader)}")

print("Finished training")

Epoch: 1, Loss: 1.3865645857690607
Epoch: 2, Loss: 0.8122970253880163
Epoch: 3, Loss: 0.5052559252792191
Epoch: 4, Loss: 0.4765114881170606
Epoch: 5, Loss: 0.46916520421944774
Epoch: 6, Loss: 0.46598978702304433
Epoch: 7, Loss: 0.4655326266311905
Epoch: 8, Loss: 0.46289343582194986
Epoch: 9, Loss: 0.463819136101644
Epoch: 10, Loss: 0.46172354681399264
Epoch: 11, Loss: 0.4577928902592637
Epoch: 12, Loss: 0.4594421913322893
Epoch: 13, Loss: 0.46020461315090216
Epoch: 14, Loss: 0.4588621584969817
Epoch: 15, Loss: 0.45374706411255
Epoch: 16, Loss: 0.4508661778815525
Epoch: 17, Loss: 0.4493555379522856
Epoch: 18, Loss: 0.4529667953843052
Epoch: 19, Loss: 0.44632875573260766
Epoch: 20, Loss: 0.4513671486412437
Epoch: 21, Loss: 0.4486653059720993
Epoch: 22, Loss: 0.4505360087434065
Epoch: 23, Loss: 0.44805664214694385
Epoch: 24, Loss: 0.4476353080527296
Epoch: 25, Loss: 0.4490152167029751
Epoch: 26, Loss: 0.4431336555026417
Epoch: 27, Loss: 0.4422840398392421
Epoch: 28, Loss: 0.44187933353094

### Getting the Q-tables and saving the data

In [7]:
prey_q_table = {t : infer(prey_net, t) for t in prey_tensors}
predator_q_table = {t : infer(predator_net, t) for t in predator_tensors}

In [12]:
with open('prey_q_table.pickle', 'wb') as f:
    pickle.dump(prey_q_table, f, protocol=pickle.HIGHEST_PROTOCOL)

with open('predator_q_table.pickle', 'wb') as f:
    pickle.dump(predator_q_table, f, protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
predator_q_table

{tensor([[2., 1., 2.],
         [2., 2., 1.],
         [0., 2., 1.]]): [1.7733609070091916e-07,
  0.49702325463294983,
  0.5029765963554382,
  7.065965257258726e-16,
  8.73541961422114e-14],
 tensor([[0., 0., 0.],
         [2., 2., 2.],
         [0., 1., 2.]]): [4.681127583000891e-18,
  4.6096644723547797e-07,
  1.3673295351859793e-12,
  0.9999995231628418,
  2.5321086034673333e-14],
 tensor([[0., 0., 0.],
         [1., 2., 2.],
         [0., 1., 2.]]): [3.1627621297879437e-17,
  2.1731817412273813e-07,
  3.47062669944255e-14,
  0.48687928915023804,
  0.5131205320358276],
 tensor([[2., 2., 2.],
         [2., 2., 0.],
         [2., 2., 2.]]): [1.835877587105794e-12,
  1.7793875747851295e-11,
  1.0,
  3.746819388084077e-08,
  5.183580853440617e-08],
 tensor([[2., 1., 1.],
         [0., 2., 2.],
         [1., 0., 2.]]): [2.9453912600341216e-36,
  0.999792754650116,
  9.967202757588112e-11,
  0.00010081531945616007,
  0.00010642889537848532],
 tensor([[2., 1., 0.],
         [2., 2., 2.],
 